In [1]:
import os
import torch.autograd
import torch.nn as nn
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader, SubsetRandomSampler
# transform = transforms.Compose([
#     transforms.Resize((64, 64)),  # 调整图像大小为统一大小
#     transforms.ToTensor(),
# ])
# # 下载和加载MNIST数据集
# train_dataset = datasets.GTSRB(root='F:\ModelAndDataset\data',split="train",transform=transform,download=True)

In [2]:
from Utils.MISC import *
selected_classes=[1,3,5,7,9,11,13]
selected_class_mapping,selected_class_name_mapping,data_loader=loadData_selected_labels(root="F:\ModelAndDataset\data",selected_classes=selected_classes,batch_size=64,train=True)

In [3]:
print(selected_class_mapping)
print(selected_class_name_mapping)


{1: 0, 3: 1, 5: 2, 7: 3, 9: 4, 11: 5, 13: 6}
{0: '30 km/h', 1: '60 km/h', 2: '80 km/h', 3: '100 km/h', 4: 'No overtaking', 5: 'Crossroad', 6: 'Give way'}


In [24]:

print(selected_class_name_mapping)
print(selected_class_name_mapping[0])
from Utils.MISC import mapping_labels
for x,y in data_loader:
    print(y)
    y=mapping_labels(selected_class_mapping,y)
    print(y)
    names=[]
    for label in y:
        print(label.item())
        print(type(label.item()))
        print(type(0))
        names.append(selected_class_name_mapping[label.item()])
    

{0: '30 km/h', 1: '60 km/h', 2: '80 km/h', 3: '100 km/h', 4: 'No overtaking', 5: 'Crossroad', 6: 'Give way'}
30 km/h
tensor([ 1, 13, 11,  9, 13, 13,  1,  1, 11,  9,  1, 13,  1,  5, 13,  3,  9,  3,
         5,  9,  1,  3, 11,  3,  7,  9,  5,  5,  1,  7, 13,  1,  5,  3, 13,  3,
        11,  9, 11,  9,  7,  5, 11,  7,  5,  9,  9,  1,  5, 13,  9,  1,  1,  5,
        13,  1,  1,  7,  3,  3, 13, 11, 13, 13])
tensor([0, 6, 5, 4, 6, 6, 0, 0, 5, 4, 0, 6, 0, 2, 6, 1, 4, 1, 2, 4, 0, 1, 5, 1,
        3, 4, 2, 2, 0, 3, 6, 0, 2, 1, 6, 1, 5, 4, 5, 4, 3, 2, 5, 3, 2, 4, 4, 0,
        2, 6, 4, 0, 0, 2, 6, 0, 0, 3, 1, 1, 6, 5, 6, 6])
0
<class 'int'>
<class 'int'>
6
<class 'int'>
<class 'int'>
5
<class 'int'>
<class 'int'>
4
<class 'int'>
<class 'int'>
6
<class 'int'>
<class 'int'>
6
<class 'int'>
<class 'int'>
0
<class 'int'>
<class 'int'>
0
<class 'int'>
<class 'int'>
5
<class 'int'>
<class 'int'>
4
<class 'int'>
<class 'int'>
0
<class 'int'>
<class 'int'>
6
<class 'int'>
<class 'int'>
0
<class 'int'>
<

In [25]:
print(names)

['Give way', '60 km/h', '30 km/h', '60 km/h', 'No overtaking', 'Give way', '30 km/h', 'Give way', '60 km/h', '60 km/h']
